<a href="https://colab.research.google.com/github/Jorayala/CiscoConnect_AI_Agents_101_Cancun_2025/blob/main/Copy_of_L2_research_write_article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [ ]:
pip install utils


  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13906 sha256=c129438b47205bbec63725d20fa71c8c9606b8258599dd1b0f1e561e01e8e75c
  Stored in directory: /root/.cache/pip/wheels/b6/a1/81/1036477786ae0e17b522f6f5a838f9bc4288d1016fc5d0e1ec
Successfully built utils


You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [ ]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [ ]:
from langchain_community.chat_models import ChatOpenAI
import os
from google.colab import userdata

#print(f"OpenAI API Key: {openai_api_key}")

planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for the "
              "Content Writer to write an article on this topic.",
    allow_delegation=False,
    verbose=True,
    llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo", api_key=userdata.get("OPENAI_API_KEY"))
)

### Agent: Writer

In [ ]:
from langchain_community.chat_models import ChatOpenAI

writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo", api_key=openai_api_key)
)

### Agent: Editor

In [ ]:
from langchain_community.chat_models import ChatOpenAI

editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo", api_key=openai_api_key)
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [ ]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [ ]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [ ]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
result = crew.kickoff(inputs={"topic": "Building the Eiffel Tower"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Building the Eiffel Tower.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: Building the Eiffel Tower

Outline:
I. Introduction
- Brief history of the Eiffel Tower
- Importance of the Eiffel Tower as a cultural icon
II. Construction of the Eiffel Tower
- Design and architecture
- Construction process and challenges faced
III. Interesting facts about the Eiffel Tower
- Height and dimensions
- Maintenance and renovations
IV. Impact of the Eiffel Tower
- Tourism and economic significance
- Cultural and historical importance
V. Call to Action
- Encourage readers to visit 

- Display the results of your execution as markdown in the notebook.

In [ ]:
from IPython.display import Markdown
Markdown(result)

# Building the Eiffel Tower: A Cultural Icon

## Introduction
The Eiffel Tower, one of the most iconic landmarks in the world, holds a significant place in history and culture. Built in 1889 as the entrance arch for the 1889 World's Fair, it was initially met with skepticism but has since become a symbol of Paris and France. The tower was named after Gustave Eiffel, the engineer whose company designed and built it. Its construction marked a turning point in architectural innovation and engineering prowess.

## Construction of the Eiffel Tower
The design and architecture of the Eiffel Tower are a testament to the creativity and vision of Gustave Eiffel and his team of engineers. Standing at 1,063 feet tall, the tower was the tallest man-made structure in the world until the completion of the Chrysler Building in New York City in 1930. The construction process was a feat of engineering, with over 18,000 individual iron parts assembled using over 2 million rivets. Despite facing challenges such as strong winds and public criticism, the Eiffel Tower was completed in just over two years.

## Interesting Facts about the Eiffel Tower
The Eiffel Tower's impressive height and dimensions make it a marvel of modern architecture. It weighs approximately 10,100 tons and has three levels that offer panoramic views of Paris. The tower undergoes regular maintenance and renovations to ensure its structural integrity and preserve its iconic status. It is estimated that over 7 million people visit the Eiffel Tower each year, making it one of the most visited paid monuments in the world.

## Impact of the Eiffel Tower
The Eiffel Tower has had a profound impact on tourism and the economy of Paris. It attracts visitors from around the globe who come to admire its beauty and learn about its history. The tower also holds cultural and historical significance as a symbol of French national pride and innovation. Its image is often used to promote tourism and represent the city of Paris in various media and marketing campaigns.

## Call to Action
Whether you are a history enthusiast, an architecture student, or a traveler looking to explore iconic landmarks, the Eiffel Tower offers a unique experience that is not to be missed. Plan a visit to this cultural icon or delve deeper into its history through reputable resources and books that provide insights into its construction, significance, and impact on the world.

In conclusion, the Eiffel Tower stands as a testament to human ingenuity and creativity, captivating audiences with its beauty and historical significance. Its construction and impact on tourism and culture make it a must-visit destination for anyone interested in exploring the wonders of the world. Visit the official website of the Eiffel Tower or read books by renowned authors to learn more about this iconic landmark.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "Origin of tea"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Origin of tea.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: Origin of Tea

Introduction:
- Brief history of tea and its significance in various cultures
- Overview of the origins of tea production and consumption

Key Points:
1. The discovery of tea: Legends and historical accounts of the first tea leaves being brewed
2. Early cultivation and trade: The spread of tea cultivation from China to other parts of Asia
3. Tea in different cultures: How tea became an integral part of traditions in countries like Japan, India, and England
4. Modern tea production: The evo

In [ ]:
Markdown(result)

# The Fascinating Origins of Tea

Tea, a beloved beverage enjoyed by millions around the world, has a rich and storied history that dates back thousands of years. From ancient legends to modern scientific research, the origins of tea are as diverse and intriguing as the cultures that have embraced it. In this blog post, we will delve into the discovery of tea, its early cultivation and trade, its significance in different cultures, the evolution of modern tea production, and the health benefits associated with this beloved drink.

## The Discovery of Tea

The story of tea's discovery is steeped in legend and historical accounts. One popular legend from China credits Emperor Shen Nong with the accidental discovery of tea around 2737 BC. According to the legend, while boiling water in the garden, some tea leaves fell into the pot, creating a fragrant and refreshing brew. This serendipitous event marked the beginning of tea consumption in China and eventually led to the spread of tea cultivation and trade throughout Asia.

## Early Cultivation and Trade

Tea cultivation began in China and gradually spread to other parts of Asia, including Japan, India, and Sri Lanka. The Silk Road played a crucial role in the trade of tea, as it was transported along with other valuable goods to distant lands. Tea became a symbol of status and wealth, with different varieties and brewing methods emerging in different regions. The art of tea drinking became an integral part of cultural traditions and social rituals in many societies.

## Tea in Different Cultures

Tea has played a significant role in shaping the traditions and customs of various cultures around the world. In Japan, the tea ceremony, known as Chanoyu, is a highly ritualized practice that emphasizes harmony, respect, purity, and tranquility. In India, chai tea is a popular beverage made with a blend of spices and milk, enjoyed by people of all ages. In England, afternoon tea became a fashionable social event, with tea served alongside sandwiches, scones, and pastries.

## Modern Tea Production

In modern times, tea production has evolved with advancements in farming and processing techniques. Sustainable practices are being implemented to ensure the quality and environmental impact of tea cultivation. From hand-picked leaves to mechanized processing, the art of tea-making continues to thrive in tea-growing regions around the world. The demand for high-quality tea has led to innovations in packaging, marketing, and distribution, making tea more accessible to a global audience.

## Health Benefits of Tea

Beyond its cultural significance, tea has been praised for its numerous health benefits. Scientific research has shown that tea contains antioxidants and other compounds that may help reduce the risk of chronic diseases such as heart disease, cancer, and diabetes. Regular consumption of tea has been linked to improved mental alertness, weight management, and overall well-being. Whether enjoyed hot or cold, tea remains a popular choice for health-conscious individuals seeking a flavorful and beneficial beverage.

In conclusion, the origin of tea is a fascinating journey that spans centuries and continents. From its mythical beginnings to its modern-day popularity, tea continues to captivate and inspire people around the world. By exploring the diverse cultures, traditions, and health benefits associated with tea, we can gain a deeper appreciation for this timeless beverage. So, why not brew a cup of your favorite tea, sit back, and savor the history and flavors that have made tea a beloved drink for generations.

*Join us on a tea-tasting adventure and discover the origins of your favorite brews. Visit TeaMuse.com for a wide selection of teas and explore the health benefits of tea at MedicalNewsToday.com. Cheers to the wonderful world of tea!*

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).

In [ ]:
import os
from google.colab import userdata

# Define a function to get the OpenAI API key
def get_openai_api_key():
  """Gets the OpenAI API key from Colab's user data."""
  try:
    return userdata.get("OPENAI_API_KEY")
  except KeyError:
    print("Warning: OPENAI_API_KEY not found in Colab secrets. Please add it to the Secrets tab.")
    return None

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

# You can add your API key to Colab's Secrets tab on the left pane.
# Name the secret 'OPENAI_API_KEY'